# Deep Learning Via ANN


Step - 1 Import Libraries

In [ ]:
import numpy as np
import pandas as pd# Importing libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
tf.__version__

'2.19.0'

Step - 2 Importing Datasets

In [ ]:
dataset = pd.read_csv('American_Express_User_Exit_Prediction.csv')
X = dataset.iloc[:, 0:-1].values
y = dataset.iloc[:, -1].values

In [ ]:
print(X)

[[553 'Delhi' 'Female' ... 4 1 274150]
 [447 'Bengaluru' 'Male' ... 4 1 519360]
 [501 'Delhi' 'Female' ... 4 1 545501]
 ...
 [627 'Mumbai' 'Female' ... 4 0 494067]
 [600 'Bengaluru' 'Female' ... 2 1 109375]
 [553 'Delhi' 'Male' ... 4 1 180031]]


Step - 3  Encode categorical data

Gender column (index 2) - Label Encoding

In [ ]:
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [ ]:
print(X)

[[553 'Delhi' 0 ... 4 1 274150]
 [447 'Bengaluru' 1 ... 4 1 519360]
 [501 'Delhi' 0 ... 4 1 545501]
 ...
 [627 'Mumbai' 0 ... 4 0 494067]
 [600 'Bengaluru' 0 ... 2 1 109375]
 [553 'Delhi' 1 ... 4 1 180031]]


Step - 4 Geography column (index 1) - OneHotEncoding

In [ ]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

# Optional: drop one dummy variable to avoid dummy variable trap (not critical for ANN)
# X = X[:, 1:]

# Split dataset into training and test sets


Step- 4 Splitting Data Set into Training and Test Set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

Step - 5 Feature Scaling

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Step - 6 Build ANN

In [ ]:
ann = tf.keras.models.Sequential()

ann.add(tf.keras.layers.Dense(units=6, activation='relu'))# Input layer + first hidden layer

ann.add(tf.keras.layers.Dense(units=6, activation='relu'))# Second hidden layer

ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))# Output layer

ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # Compile ANN

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)# Early stopping_
#__to prevent overfitting



Step - 7  Train ANN

In [ ]:
ann.fit(X_train, y_train, validation_split=0.2, batch_size=32, epochs=120, callbacks=[early_stop])

Epoch 1/120
199/199 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6560 - loss: 0.6598 - val_accuracy: 0.8011 - val_loss: 0.5146
Epoch 2/120
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7880 - loss: 0.5138 - val_accuracy: 0.8074 - val_loss: 0.4677
Epoch 3/120
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7954 - loss: 0.4725 - val_accuracy: 0.8068 - val_loss: 0.4476
Epoch 4/120
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7885 - loss: 0.4630 - val_accuracy: 0.8074 - val_loss: 0.4356
Epoch 5/120
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7952 - loss: 0.4461 - val_accuracy: 0.8049 - val_loss: 0.4292
Epoch 6/120
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7946 - loss: 0.4381 - val_accuracy: 0.8112 - val_loss: 0.4255
Epoch 7/120
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7985 - loss: 0.4328 - val_accuracy: 0.8162 - val_loss: 0.4214
Epoch 8/120
199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8035 - loss: 0.4281 - val_accu

Step - 8  Prediction on test set

In [ ]:

y_pred = ann.predict(X_test)
y_pred_bool = (y_pred > 0.5)



63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


Step - 9 Confusion Matrix and Accuracy

In [ ]:
cm = confusion_matrix(y_test, y_pred_bool)
acc = accuracy_score(y_test, y_pred_bool)
print("Confusion Matrix:\n", cm)
print("Test Accuracy:", acc)

Confusion Matrix:
 [[1526   47]
 [ 238  175]]
Test Accuracy: 0.8564954682779456


Step - 10  Single Prediction (False Case)

In [ ]:
# Make sure input matches the post-encoding feature count
# Example: adjust the values according to the dataset columns
single_input = np.array([[0, 1, 0, 501, 0, 32, 2, 0, 4, 1, 545501]])  # match X_train columns
single_input_scaled = sc.transform(single_input)
single_prediction = ann.predict(single_input_scaled) > 0.5
print("Single Prediction (True=Exit, False=Stay):", single_prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Single Prediction (True=Exit, False=Stay): [[False]]


Step - 11  Single Prediction (True Case)

In [ ]:
single_input = np.array([[0, 1, 0, 400, 1, 58, 1, 0, 0, 0, 80000]])
single_input_scaled = sc.transform(single_input)
prediction = ann.predict(single_input_scaled) > 0.5
print("Exit prediction (True=Exit, False=Stay) :", prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Exit prediction (True=Exit, False=Stay) : [[ True]]
